# AI Video GPU Platform - SageMaker Studio Lab Setup

This notebook sets up the AI Video GPU platform on Amazon SageMaker Studio Lab with free GPU access.

## Prerequisites
- SageMaker Studio Lab account (free, but requires application approval)
- Email verification
- GPU runtime selected

## GPU Availability
- **Free Tier**: Up to 4 hours/day of GPU time
- **GPU Types**: T4 (single GPU)
- **RAM**: 12GB
- **Storage**: 15GB persistent
- **CPU**: 4 vCPUs

⚠️ **Important**: Make sure to select GPU runtime when starting your project

## Step 1: System Information & GPU Check

In [ ]:
import os
import sys
import subprocess
import torch

print("🔍 System Information:")
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")
    
    # Check GPU utilization
    !nvidia-smi
else:
    print("⚠️ GPU not detected!")
    print("Make sure you're using a GPU runtime:")
    print("1. Go to File → New → Project")
    print("2. Select 'GPU' as the compute type")
    print("3. Or restart with GPU runtime")

# Check available disk space
result = subprocess.run(['df', '-h', '/home/studio-lab-user'], capture_output=True, text=True)
print(f"\n💾 Disk space:\n{result.stdout}")

## Step 2: Install System Dependencies

In [ ]:
# Install system packages (SageMaker Studio Lab has conda preinstalled)
!conda install -y -c conda-forge ffmpeg

# Install additional system packages if needed
!pip install --upgrade pip

print("✅ System dependencies installed successfully!")

## Step 3: Clone Repository

In [ ]:
import os

# Change to home directory
os.chdir('/home/studio-lab-user')

# Clone the repository
if not os.path.exists('AI-Video-GPU'):
    !git clone https://github.com/yourusername/AI-Video-GPU.git
    print("✅ Repository cloned successfully!")
else:
    print("📁 Repository already exists")
    # Update existing repository
    os.chdir('AI-Video-GPU')
    !git pull origin main
    os.chdir('..')

# Change to project directory
os.chdir('/home/studio-lab-user/AI-Video-GPU')
print(f"📍 Current directory: {os.getcwd()}")

# List contents to verify
!ls -la

## Step 4: Install Python Dependencies

In [ ]:
# Install main requirements
!pip install -r requirements.txt

# Install additional packages for SageMaker Studio Lab
!pip install gradio ipywidgets

# Enable widget extensions for Jupyter
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

print("✅ Python dependencies installed successfully!")

## Step 5: Configure Environment

In [ ]:
import os

# Set environment variables for SageMaker Studio Lab
os.environ['SAGEMAKER_ENV'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Single GPU
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# Set paths (using persistent storage)
os.environ['PROJECT_ROOT'] = '/home/studio-lab-user/AI-Video-GPU'
os.environ['MODEL_CACHE'] = '/home/studio-lab-user/models'
os.environ['OUTPUT_DIR'] = '/home/studio-lab-user/outputs'

# Create necessary directories
os.makedirs('/home/studio-lab-user/models', exist_ok=True)
os.makedirs('/home/studio-lab-user/outputs', exist_ok=True)
os.makedirs('/home/studio-lab-user/temp', exist_ok=True)

print("✅ Environment configured for SageMaker Studio Lab!")
print(f"📁 Models cache: {os.environ['MODEL_CACHE']}")
print(f"📁 Output directory: {os.environ['OUTPUT_DIR']}")
print("💾 Note: Files are persistent across sessions!")

## Step 6: Download Models (Optional - Heavy Downloads)

In [ ]:
# This cell downloads AI models. Uncomment and run if needed.
# Note: Models are large and may take significant time
# Advantage: Models persist across sessions in SageMaker Studio Lab

# import torch
# from huggingface_hub import snapshot_download
# import os
# 
# model_cache = '/home/studio-lab-user/models'
# 
# print("🔄 Downloading models... This may take several minutes")
# print("💾 Models will be cached and persist across sessions")
# 
# # Download Stable Diffusion model
# try:
#     model_path = snapshot_download(
#         repo_id="runwayml/stable-diffusion-v1-5",
#         cache_dir=model_cache,
#         allow_patterns=["*.bin", "*.json", "*.txt"]
#     )
#     print(f"✅ Stable Diffusion downloaded to {model_path}")
# except Exception as e:
#     print(f"⚠️ Error downloading Stable Diffusion: {e}")
# 
# # Check disk usage after download
# !df -h /home/studio-lab-user

print("💡 Model download cell ready. Uncomment code above to download models.")
print("✅ Advantage: Models persist across sessions in Studio Lab!")

## Step 7: Launch the Application

In [ ]:
import sys
import os

# Add project to Python path
sys.path.append('/home/studio-lab-user/AI-Video-GPU')

try:
    # Import and launch the main application
    from main import create_gradio_interface
    
    # Create the Gradio interface
    demo = create_gradio_interface()
    
    # Launch with SageMaker Studio Lab specific settings
    demo.launch(
        share=True,  # Creates public link
        server_name="0.0.0.0",
        server_port=7860,
        debug=True,
        show_error=True
    )

except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Running basic demo instead...")
    
    # Fallback: Basic Gradio demo
    import gradio as gr
    
    def basic_demo(text):
        gpu_info = ""
        if torch.cuda.is_available():
            gpu_info = f"GPU: {torch.cuda.get_device_name(0)}"
        return f"AI Video GPU Platform is running on SageMaker Studio Lab! {gpu_info}\nInput: {text}"
    
    def gpu_test():
        if torch.cuda.is_available():
            # Simple GPU test
            device = torch.device('cuda')
            x = torch.randn(1000, 1000, device=device)
            y = torch.randn(1000, 1000, device=device)
            z = torch.matmul(x, y)
            return f"✅ GPU test passed! Result shape: {z.shape}"
        else:
            return "❌ No GPU available"
    
    with gr.Blocks(title="AI Video GPU - SageMaker Studio Lab") as demo:
        gr.Markdown("# AI Video GPU Platform - SageMaker Studio Lab Demo")
        gr.Markdown("Basic demo running on SageMaker Studio Lab GPU environment")
        
        with gr.Row():
            with gr.Column():
                text_input = gr.Textbox(label="Test Input", placeholder="Enter test text...")
                test_btn = gr.Button("Test Platform")
                gpu_test_btn = gr.Button("Test GPU")
            
            with gr.Column():
                output = gr.Textbox(label="Output")
                gpu_output = gr.Textbox(label="GPU Test Result")
        
        test_btn.click(basic_demo, inputs=text_input, outputs=output)
        gpu_test_btn.click(gpu_test, outputs=gpu_output)
    
    demo.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860
    )

## Step 8: Monitor Resources

In [ ]:
import torch
import psutil
import time

def monitor_resources():
    print("📊 Resource Monitoring:")
    print("=" * 50)
    
    # CPU and Memory
    cpu_percent = psutil.cpu_percent(interval=1)
    memory = psutil.virtual_memory()
    print(f"CPU Usage: {cpu_percent}%")
    print(f"Memory Usage: {memory.percent}% ({memory.used / 1024**3:.2f} GB / {memory.total / 1024**3:.2f} GB)")
    
    # Disk usage
    disk = psutil.disk_usage('/home/studio-lab-user')
    print(f"Disk Usage: {disk.used / 1024**3:.2f} GB / {disk.total / 1024**3:.2f} GB ({disk.used / disk.total * 100:.1f}%)")
    
    # GPU info
    if torch.cuda.is_available():
        print(f"\n🎮 GPU Status:")
        print(f"GPU Memory Allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
        print(f"GPU Memory Cached: {torch.cuda.memory_reserved()/1024**3:.2f} GB")
        
        # GPU utilization
        !nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total --format=csv,noheader,nounits
    
    print("=" * 50)

# Run monitoring
monitor_resources()

# Set up periodic monitoring (optional)
print("\n💡 Run this cell periodically to monitor resource usage")
print("⚠️ Remember: You have 4 hours/day of GPU time in Studio Lab")

## Step 9: Save and Manage Files

In [ ]:
import os
import shutil
from datetime import datetime

def create_backup():
    """Create a backup of important files"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_dir = f"/home/studio-lab-user/backups/backup_{timestamp}"
    
    os.makedirs(backup_dir, exist_ok=True)
    
    # Backup outputs
    if os.path.exists('/home/studio-lab-user/outputs'):
        shutil.copytree('/home/studio-lab-user/outputs', f"{backup_dir}/outputs")
        print(f"✅ Outputs backed up to {backup_dir}/outputs")
    
    # Backup any custom configs
    config_files = ['config.json', 'settings.yaml', '.env']
    for config_file in config_files:
        if os.path.exists(f'/home/studio-lab-user/AI-Video-GPU/{config_file}'):
            shutil.copy2(f'/home/studio-lab-user/AI-Video-GPU/{config_file}', backup_dir)
            print(f"✅ {config_file} backed up")
    
    return backup_dir

def list_files():
    """List important files and their sizes"""
    print("📁 File Management:")
    print("=" * 50)
    
    # List outputs
    if os.path.exists('/home/studio-lab-user/outputs'):
        print("\n📤 Output Files:")
        for root, dirs, files in os.walk('/home/studio-lab-user/outputs'):
            for file in files:
                filepath = os.path.join(root, file)
                size = os.path.getsize(filepath) / 1024**2  # MB
                print(f"  {file}: {size:.2f} MB")
    
    # List models
    if os.path.exists('/home/studio-lab-user/models'):
        print("\n🤖 Cached Models:")
        for root, dirs, files in os.walk('/home/studio-lab-user/models'):
            for file in files:
                if file.endswith(('.bin', '.pt', '.pth', '.safetensors')):
                    filepath = os.path.join(root, file)
                    size = os.path.getsize(filepath) / 1024**3  # GB
                    print(f"  {file}: {size:.2f} GB")

# Run file management functions
list_files()

print("\n💡 Use create_backup() to backup your work")
print("💾 Files in /home/studio-lab-user persist across sessions")

# Uncomment to create backup
# backup_path = create_backup()
# print(f"Backup created at: {backup_path}")

## Usage Tips for SageMaker Studio Lab

### 📝 Important Notes:
1. **GPU Quota**: 4 hours/day of free GPU time
2. **Persistent Storage**: 15GB that persists across sessions
3. **Session Management**: Sessions can run for up to 8 hours
4. **Account**: Requires approval (may take 1-3 days)

### 💡 Best Practices:
- Save frequently - your work persists!
- Monitor GPU time usage carefully
- Use conda environments for package management
- Cache models in persistent storage
- Stop GPU sessions when not in use

### 🔧 Troubleshooting:
- **No GPU**: Make sure you selected GPU runtime when creating project
- **Out of GPU Time**: Wait until next day for quota reset
- **Out of Storage**: Clean up old models and outputs
- **Package Issues**: Use conda instead of pip when possible

### 📊 Resource Management:
```python
# Check GPU time remaining
# (View in Studio Lab dashboard)

# Monitor disk usage
!df -h /home/studio-lab-user

# Clean up models
!du -sh /home/studio-lab-user/models/*

# Check GPU status
!nvidia-smi
```

### 🚀 Performance Tips:
- Use mixed precision training (fp16)
- Enable gradient checkpointing for large models
- Batch smaller requests together
- Cache frequently used models
- Use smaller model variants for testing

## 🎉 Setup Complete!

Your AI Video GPU platform is now running on SageMaker Studio Lab!

### Key Advantages:
- ✅ **Persistent Storage**: Your models and outputs are saved
- ✅ **Professional Environment**: Full Jupyter Lab interface
- ✅ **Stable Performance**: Dedicated T4 GPU
- ✅ **No Time Limits**: 8-hour sessions (vs 90 min in Colab)

### Next Steps:
1. Access the Gradio interface above
2. Upload your media files
3. Configure generation settings
4. Generate AI videos and content
5. Your work will be saved automatically!

### Resource Management:
- Monitor GPU time in the Studio Lab dashboard
- Clean up old files to manage the 15GB storage limit
- Stop GPU sessions when not actively using them

Happy creating! 🚀